## 1. Pattern Detection

### 1.0 Retrieve the names of all ast files

In [ ]:
from code_analysis import AST, ASTReader
from pathlib import Path

### Get all the filenames in a given directory that sastifies a condition (*.ast.json)
def get_ast_json_files(directory):
   directory = Path(directory)
   return [str(p) for p in directory.rglob('*.ast.json')]

### 1.1 Create visitor that will visit a given ast in search of calls to an SQL Database
_**Note:** In php, calls to an SQL db can be done in the following ways:_
```
    mysql_query ( * ) ;
    mysqli_query ( * ) ;
    $object - > execute () ;
    $object - > mysql - > exec ( * ) ;
```

In [ ]:
class ASTSqlCallVisitor:
    def visit(self, ast: AST):
        self.ast = ast
        print(f"----- Visiting AST from file {self.ast.get_filename()} -----")
        # Start by visiting the root node of the ast
        # The private 'visit function' will recursively visit the children of each node
        self.__visit(self.ast.get_root())
        
    def __visit(self, node_id: int):
        # Define the conditions that must be respected for a node to be considered a call to an SQL DB (based on the above note)
        nodeImage = self.ast.get_image(node_id)
        isMethodOrFunctionCall = (self.ast.get_type(node_id) == "FunctionCall" or self.ast.get_type(node_id) == "MethodCall")

        isSqlQuery = isMethodOrFunctionCall and nodeImage == "mysql_query"
        isSqliQuery = isMethodOrFunctionCall and nodeImage == "mysqli_query"
        isExecuteMethodCall = isMethodOrFunctionCall and nodeImage == "execute"
        isExecMethodCall = isMethodOrFunctionCall and nodeImage == "exec

        if isSqlQuery or isSqliQuery or isExecuteMethodCall or isExecMethodCall:
            print(f"Function '{nodeImage}' is called "
            f"at line {self.ast.get_position(node_id)[0]}")



#   def visit(self, ast: AST):
#     self.ast = ast
#     print(f"Visit AST from file {self.ast.get_filename()}")
#     self.__visit(self.ast.get_root())

#   def __visit(self, node_id: int):
#     if self.ast.get_type(node_id) == "FunctionCall":
#       print(f"Function '{self.ast.get_image(node_id)}' is called "
#             f"at line {self.ast.get_position(node_id)[0]}")

#     for child_id in self.ast.get_children(node_id):
#       self.__visit(child_id)

# visitor = ASTFunctionCallVisitor()
# visitor.visit(ast)
# visitor.visit(ast_2)

### 1.2 For each file, load ast in memory and visit nodes in search of db calls  

## 2. Known vulnerability detection

